# Network Performance Planner Installation guidance
This notebook outlines the setup and usage of the Network Performance Planner blueprint, which leverages an agentic AI workflow for dynamic telecom network optimization.

## Step 1: Clone the Repository

In [ ]:
!git clone https://github.com/bubbleran/Telco-Network-Configuration.git

## Step 2: Set Up Environment Variables
Before running any services, export your NVIDIA API key and prepare the local cache. If you do not have the NVIDIA API key, please follow these steps to create one:
1. Visit build.nvidia.com
2. Create an account
3. Click your account icon in the screen upper corner
4. Select API keys from the menu
5. Press Generate API key

Once you have a valid API key, edit the /Telco-Network-Configuration/config.yaml  and add the NVIDIA API KEY.
Also review the other configuration parameters for the blueprint.

In [1]:
import yaml
import os
from os.path import expanduser

NGC_API_KEY=yaml.safe_load(open('Telco-Network-Configuration/config.yaml', 'r'))['nvidia_api_key']
os.environ["NGC_API_KEY"] = NGC_API_KEY

## Step 3: Run NIM Container (Optional - for Local NIM Hosting)

In [2]:
os.environ["LOCAL_NIM_CACHE"]=expanduser("~/.cache/nim")
!mkdir -p {os.environ["LOCAL_NIM_CACHE"]}
!sudo chmod -R 777 {os.environ["LOCAL_NIM_CACHE"]}

!echo $NGC_API_KEY | docker login -u '$oauthtoken' --password-stdin nvcr.io 

!docker run -d --rm \
    --name llama3.1-70b-container \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY={os.environ["NGC_API_KEY"]} \
    -v {os.environ["LOCAL_NIM_CACHE"]}:/opt/nim/.cache \
    -p 8000:8000 \
    nvcr.io/nim/meta/llama-3.1-70b-instruct:latest


WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores

Login Succeeded
f8622075bdce89008a5037f843c70a0566962db146f66bb088344944f0045e85


It might take 5-10 minutes to setup the LLama-3.1-70b. Monitor the status of the NIM using the docker logs by running the command below. You can rerun the following command to check the status. Once it is ready, it should output `Uvicorn running on http://0.0.0.0:8000`

In [7]:
!docker logs --tail 10 llama3.1-70b-container

INFO 2025-06-09 18:14:59.943 api_server.py:222] Route: /v1/health/live, Methods: GET
INFO 2025-06-09 18:14:59.943 api_server.py:222] Route: /v1/health/ready, Methods: GET
INFO 2025-06-09 18:14:59.943 api_server.py:222] Route: /v1/metrics, Methods: GET
INFO 2025-06-09 18:14:59.943 api_server.py:222] Route: /v1/license, Methods: GET
INFO 2025-06-09 18:14:59.943 api_server.py:222] Route: /v1/metadata, Methods: GET
INFO 2025-06-09 18:14:59.943 api_server.py:222] Route: /v1/manifest, Methods: GET
INFO 2025-06-09 18:14:59.988 server.py:82] Started server process [202]
INFO 2025-06-09 18:14:59.988 on.py:48] Waiting for application startup.
INFO 2025-06-09 18:14:59.992 on.py:62] Application startup complete.
INFO 2025-06-09 18:14:59.994 server.py:214] Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


To update the config.yaml file to use endpoints of this locally hosted NIMs instead of API catalog endpoints, set `NIM_mode: True`.

## Step 4: Set Up Python Environment

In [8]:
!pip install virtualenv
!python3.10 -m virtualenv bp_env
!source bp_env/bin/activate && pip install -r Telco-Network-Configuration/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
created virtual environment CPython3.10.12.final.0-64 in 128ms
  creator CPython3Posix(dest=/home/ubuntu/bp_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/home/ubuntu/.local/share/virtualenv)
    added seed packages: GitPython==3.1.44, MarkupSafe==3.0.2, PyYAML==6.0.2, aiohappyeyeballs==2.6.1, aiohttp==3.12.11, aiosignal==1.3.2, altair==5.5.0, annotated_types==0.7.0, anyio==4.9.0, async_timeout==4.0.3, attrs==25.3.0, blinker==1.9.0, cachetools==5.5.2, certifi==2025.4.26, charset_normalizer==3.4.2, click==8.2.1, dataclasses_json==0.6.7, exceptiongroup==1.3.0, filetype==1.2.0, frozenlist==1.6.2, gitdb==4.0.12, greenlet==3.2.3, h11==0.16.0, httpcore==1.0.9, httpx==0.28.1, httpx_sse==0.4.0, idna==3.10, jinja2==3.1.6, jsonpatch==1.33, jsonpointer==3.0.0, jsonschema==4.24.0, jsonschema_specifications==2025.4.1, langcha

## Step 5: Start the Streamlit UI

In [9]:
!source bp_env/bin/activate && cd Telco-Network-Configuration && streamlit run telco_planner_ui.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.27.30.232:8501
  External URL: http://204.52.25.162:8501

Successfully created database './data/historical_db' and populated it with historical KPI data.



UPDATING p0_nominal tp -90 in gnb utils
Updated 'p0_nominal' to '-90' in gNodeB configuration file: 5g-sa-nr-sim/mx-conf/gnb.conf
Network not running. Reading default p0_nominal from config



UPDATING dl_carrierBandwidth tp 51 in gnb utils
Updated 'dl_carrierBandwidth' to '51' in configuration file: 5g-sa-nr-sim/mx-conf/gnb.conf
Updated 'dl_carrierBandwidth' to '51' in configuration file: 5g-sa-nr-sim/mx-conf/nr-rfsim.conf
Network not running. Reading default dl_carrierBandwidth from config



UPDATING ul_carrierBandwidth tp 51 in gnb utils
Updated 'ul_carrierBandwidth' to '51' in configuration file: 5g-sa-nr-sim/mx-conf/gnb.conf
Updated 'ul_carrierBandwidth' to '51' in configuration file: 5g-sa-nr-sim/mx-conf/nr

## Step 6: Reset the Environment (Optional)

To stop the application, please stop the kernel of the above cell.

In [14]:
!bash ./Telco-Network-Configuration/agentic_llm_workflow/reset.sh

./Telco-Network-Configuration/agentic_llm_workflow/reset.sh: line 11: cd: 5g-sa-nr-sim/: No such file or directory
no configuration file provided: not found
./Telco-Network-Configuration/agentic_llm_workflow/reset.sh: line 15: cd: ../5g-sa-usrp/: No such file or directory
no configuration file provided: not found
rm: cannot remove '../data/persistent_db': No such file or directory
rm: cannot remove '../data/historical_db': No such file or directory
Cleanup complete


In [15]:
!docker stop llama3.1-70b-container

llama3.1-70b-container


## Notes:
- This blueprint supports BubbleRAN simulation or USRP-based real-device environments.
- The agentic system includes Config, Validation, and Monitoring agents for continuous network optimization.